In [ ]:
! pip install transformers

In [ ]:
from transformers import AutoTokenizer, TapasForQuestionAnswering
import pandas as pd

# 토크나이저 및 모델 정의
tokenizer = AutoTokenizer.from_pretrained(
    "google/tapas-base-finetuned-wtq")
model = TapasForQuestionAnswering.from_pretrained(
    "google/tapas-base-finetuned-wtq")

# 테이블 데이터 정의
data = {
    "Company": ["Apple", "Microsoft", "Google"],
    "CEO": ["Tim Cook", "Satya Nadella", "Sundar Pichai"],
    "Headquarters": ["Cupertino", "Redmond", "Mountain View"]
}
# 데이터를 판다스 데이터프레임으로 변환
table = pd.DataFrame.from_dict(data)

# 쿼리(질의) 정의
queries = ["Who is the CEO of Microsoft?",
           "Where is the headquarters of Google?"]

# 테이블 및 쿼리 토큰화
inputs = tokenizer(table=table, queries=queries, padding="max_length",
                   return_tensors="pt")
# 모델을 통해 예측 수행
outputs = model(**inputs)

# 예측된 답이 담긴 셀 좌표와 집계 연산의 인덱스 추출
predicted_answer_coordinates, predicted_aggregation_indices = \
tokenizer.convert_logits_to_predictions(
    inputs,
    outputs.logits.detach(),
    outputs.logits_aggregation.detach()
)

# 쿼리 집합(queries)에서 쿼리(query)를 하나씩 꺼내 반복 수행 후 답을 출력
for i, query in enumerate(queries):
    if predicted_aggregation_indices[i] == 0:
        # 만약 집계 연산이 없을 경우(index 0), 답이 담긴 셀을 출력
        coords_to_answer = ' '.join([table.iat[coord] for coord
                                     in predicted_answer_coordinates[i]])
        print(f"Question: {query}")
        print(f"Answer: {coords_to_answer}\n")
    else:
        # 만약 집계 연산이 있다면, 연산 집합 리스트에서 집계 연산 작업명을 출력
        print(f"Question: {query}")
        print(
            f"Answer: "
            f"{tokenizer.model.config.id2label[predicted_aggregation_indices[i]]}\n"
            )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/490 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/262k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


Question: Who is the CEO of Microsoft?
Answer: Satya Nadella

Question: Where is the headquarters of Google?
Answer: Mountain View

